## Australian Rainfall Data Download and Exploratory Data Analysis

Attribution: code adapted from DSCI 525 lecture notes.

#### 1. Imports 

In [24]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
from memory_profiler import memory_usage
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.feather as feather
import rpy2
import rpy2.rinterface

#### 2. Downloading data using figshare API

In [2]:
# Necessary metadata
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text) 
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 5.65 s, sys: 6.73 s, total: 12.4 s
Wall time: 1min 57s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 18.1 s, sys: 2.74 s, total: 20.8 s
Wall time: 22 s


#### 3. Combining data files into one CSV file

In [7]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r"[\/|\\](.*)_daily_rainfall", file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

CPU times: user 5min 42s, sys: 21.8 s, total: 6min 4s
Wall time: 6min 18s


In [8]:
%%sh
du -sh figsharerainfall/combined_data.csv

5.6G	figsharerainfall/combined_data.csv


In [9]:
%%time
df.head()

CPU times: user 170 µs, sys: 284 µs, total: 454 µs
Wall time: 461 µs


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


##### 3.1 Comparing run times on different machines 

- We used pandas to concatenate the files in data.zip folder to a combined file which is `5.6GB` in size.
- The run time on different machines ranged from `6 min to 10 min` across different machines (one MacOS vs two Windows 10 operating system). The MacOS with 16GB RAM took the least time to combine the datafile into run large csv file.
- Runtime observations on group members' laptops are shown below:

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken to combine files |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Arlin   |      MacOS Monterey|  16GB   | 2 GHz Quad-Core Intel Core i5          |    No    |  Total: 6 min 9s; Wall time: 6 min 20s
| Artan   |      Windows 10    |  8GB   |   1.80 GHz Intel Core i5                |    Yes    |       Total: 9min 4s; Wall time: 9min 28s      |
| Shi Yan |      Windows 10    |  8GB   |   1.50 GHz Intel Core i5                |    Yes    |       Total: 10min 23s; Wall time: 10min 11s      |


#### 4. Load the combined CSV to memory and perform a simple EDA

EDA question: Perform value counts on a float variable `lat_min`.

##### 4.1 Regular Loading

In [10]:
%%time
# Load time
df = pd.read_csv("figsharerainfall/combined_data.csv")

CPU times: user 53 s, sys: 21.9 s, total: 1min 14s
Wall time: 1min 27s


In [11]:
%%time
# Simple EDA
counts = df["lat_min"].value_counts()

CPU times: user 762 ms, sys: 470 ms, total: 1.23 s
Wall time: 1.53 s


##### 4.2 Changing Data Type

In [12]:
%%time
# Load time
df_16 = pd.read_csv("figsharerainfall/combined_data.csv", dtype={"lat_min": np.float16,
                                                              "lat_max": np.float16,
                                                              "lon_min": np.float16,  
                                                              "lon_max": np.float16,
                                                              "rain (mm/day)": np.float16})

CPU times: user 50.9 s, sys: 11.2 s, total: 1min 2s
Wall time: 1min 7s


In [13]:
# Comparing memory space between datatype64GB and 16GB
print(f"DataFrame with numeric columns as float16: {df_16.memory_usage().sum() / 1e6:.2f} GB")
print(f"DataFrame with numeric columns as float64: {df.memory_usage().sum() / 1e6:.2f} GB")

DataFrame with numeric columns as float16: 1624.16 GB
DataFrame with numeric columns as float64: 3498.20 GB


In [14]:
%%time
# Simple EDA
counts = df_16["lat_min"].value_counts()

CPU times: user 739 ms, sys: 202 ms, total: 941 ms
Wall time: 1.02 s


##### 4.3 Load Selected Columns Only

In [15]:
%%time
# Load time
df = pd.read_csv("figsharerainfall/combined_data.csv", usecols=["lat_min","lat_max", "rain (mm/day)"])

CPU times: user 29.6 s, sys: 8.88 s, total: 38.4 s
Wall time: 42.4 s


In [16]:
%%time
# Simple EDA
counts = df["lat_min"].value_counts()

CPU times: user 775 ms, sys: 365 ms, total: 1.14 s
Wall time: 1.35 s


##### 4.4 Loading in Chunks

In [17]:
%%time
# Load and EDA time
chunk_counts = pd.Series(dtype=np.float16)
for chunk in pd.read_csv("figsharerainfall/combined_data.csv", chunksize=10_000_000):
    chunk_counts.add(chunk["lat_min"].value_counts(), 
                     fill_value=None)  # To exclude NaN
chunk_counts.head()

CPU times: user 52.6 s, sys: 9.88 s, total: 1min 2s
Wall time: 1min 4s


Series([], dtype: float16)

##### 4.5 Loading with Dask

In [18]:
%%time
# Load time
import dask.dataframe as dd
df_dask = dd.read_csv("figsharerainfall/combined_data.csv")

CPU times: user 395 ms, sys: 284 ms, total: 679 ms
Wall time: 1.83 s


In [19]:
%%time
#computing values of lat_min variable 
df_dask["lat_min"].value_counts().compute()

CPU times: user 37.5 s, sys: 12.1 s, total: 49.7 s
Wall time: 13.7 s


-31.099476    3035329
-32.984293    3035329
-34.869110    3035329
-32.041885    3035329
-30.000000    1747830
               ...   
-30.696652     183960
-36.277805     183960
-36.281964     183960
-30.700015     183960
-33.487232     183960
Name: lat_min, Length: 84, dtype: int64

##### 4.6 Comparing Wall Time of Different Machines

| Team Member | Task  |  Regular Loading |      Changed Data Type |      Selected Columns |  Chunks            |   Dask        |
|:-----------:|:------|:----------------:|:----------------------:|:---------------------:|:------------------:|:-------------:|
| Arlin       | Load  |       1 min 24s        |          1 min 10s         |           40.4s         |load + EDA:1min 6s    |     1.37s   | 
|             | EDA   |       1.4s      |             944ms      |            624ms      |included above      |      14.1s    |
| Artan       | Load  |    1min 34s      |        1min 19s        |       53.2 s          |load + EDA: 1min 14s|    766 ms     |
|             | EDA   |       1.25 s     |        1.05 s          |       766 ms          |included above      |    1min 21s   |
| Shi Yan     | Load  |    2min 33s      |        2min 12s        |       1min 27s        |load + EDA: 3min 14s|    23.35 s    |
|             | EDA   |       7.14 s     |         14.03 s        |       9.76 s          |included above      |    2min 3s    |

>Observations: Artan's laptop did not have problem loading the csv files but was getting errors like the below when doing an EDA including calculating mean values per model:
>```Python
>MemoryError: Unable to allocate 477. MiB for an array with shape (1, 62467843) and data type object
>```


>Shi Yan's laptop got the same memory error, operations also took longer time.

- Compared with the original dataframe float64 datatype, changing to float16 reduced the memory space of the dataframe from 3498.20 GB to 1624.16GB. Performing value_counts() EDA on the column `lat_min` was slightly faster for float16 than for float64 (1.02s vs. 1.53s respectively)
- Doubling the RAM size from 8GB to 16GB would boost the performance more than two times.
- **summary about dask vs. regular loading in pandas***

#### 5. Perform a simple EDA in R

In [20]:
# Limit dataset to cover two models with time-saving
%time
df_red = pd.read_csv("figsharerainfall/combined_data.csv", nrows=2000000)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [30]:
# Load the ipython extension that provides the %%R cell magic
# %load_ext rpy2.ipython
%load_ext memory_profiler

##### 5.1 Transfer the dataframe from python to R

##### Pandas Exchange

In [ ]:
%%time
%%R -i df_red
%memit
start_time <- Sys.time()
result <- df_red
head(df_red)
end_time <- Sys.time()
print(end_time - start_time)

##### Parquet file

In [31]:
%%time
%memit
df.to_parquet("figsharerainfall/combined_data.parquet")

peak memory: 2295.87 MiB, increment: 0.14 MiB
CPU times: user 4.62 s, sys: 1.61 s, total: 6.23 s
Wall time: 8.77 s


In [32]:
%%R
suppressMessages(library(arrow, warn.conflicts = FALSE))
suppressMessages(library(dplyr, warn.conflicts = FALSE))
suppressMessages(library(tidyverse, warn.conflicts = FALSE))

UsageError: Cell magic `%%R` not found.


In [33]:
%%time
%%R
ds <- open_dataset("figsharerainfall/combined_data.parquet")
so <- Scanner$create(ds)
at <- so$ToTable()
df <- as.data.frame(at)
head(df)

UsageError: Cell magic `%%R` not found.


##### Simple EDA in R with parquet

In [ ]:
%%R
# To see models covered by the dataset
models <- unique(df$model)
models

In [ ]:
%%R
# Wrangling
df$time <- as.Date(df$time)
df$year <- format(as.Date(df$time, format = "%d/%m/%Y"),"%Y")
# Only cover two models for comparison
df <- subset(df, (model == "MPI-ESM-1-2-HAM" | model == "AWI-ESM-1-1-LR"))
df_year <- aggregate(df$`rain (mm/day)`, by=list(df$year, df$model), FUN = sum)
names(df_year)[names(df_year) == "Group.1"] <- "year"
names(df_year)[names(df_year) == "Group.2"] <- "model"
names(df_year)[names(df_year) == "x"] <- "rain"
head(df_year)

In [ ]:
%%R -w 800
# To compare trendlines of two models
ggplot(df_year, aes(x = year, y = rain, color = model, group=1)) +
    geom_line() + labs(y='Rain of the year', x='Year') +
    scale_x_discrete(breaks=c(1889, 1914, 1939, 1964, 1989, 2014)) +
    ggtitle("Comparison of two models on Rainfall from 1889 to 2014")

##### feather format

In [34]:
%%R
library(arrow)
library(dplyr)

UsageError: Cell magic `%%R` not found.


In [ ]:
# load in combined dataset again as df
df = pd.read_csv("figsharerainfall/combined_data.csv")

In [ ]:
# Convert df to table with pandas 
%%time
table = pa.Table.from_pandas(df)

In [ ]:
%%time
# Write to feather format 
feather.write_feather(table, "figsharerainfall/combined.feather")

In [ ]:
# file size with feather
%%sh
du -sh figsharerainfall/combined.feather

##### Simple EDA with feather format

In [ ]:
%%time
%%R

start_time <- Sys.time()
r_table <- arrow::read_feather("figsharerainfall/combined.feather")
print(class(r_table))

result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

**Observation Summary - Windows10 laptop (Shi Yan)**

|     Approach    | CPU Time  | Wall Time | file size| Peak Memory (MiB)|
|:---------------:|:----------|:---------:|:---------: |:---------:|
| Pandas Exchange | 1min 24s  |  1min 25s |    -      |          |
| Parquet file    |  1.55 s   |    1.78 s |          |   2295.87       |
| feather format  |           |           |     1.2 GB    |          |

Three approaches are explored here: `Pandas Exchange`,`Parquet file` and `feather format`

Both methods have its own advantages:

`Pandas exchange` - Easy and simple, the goal can be achieved by one-line code. Total wall time it took on Windows10 was around 1 min 25s. Most of the time is spent on serialization and deserialization process. 

`Parquet file` - Require more code lines but not heavy. The benefit of running time is significant. it has the benefits of predicate and projection pushdown. Since it has a hybrid model of column-based and row-based file storage properties, we were able to split the `combined_data.csv` file to selected rows. With the small size we are saving some I/O and network traffic.  

`feather format` - The file size of feather format is quite small and sufficient. It also has a faster runtime than parquet. The file space only takes about 1.2GB in memory compared to the 5.6 GB with csv we saw earlier with pandas exchange. While it may be a solution for short term exhange, parquet might be more efficient for long term storage. 
<br><br>

There is another approach called `Arrow exchange` which are similar to `Parquet file`. 
`Arrow exchange` is more suitable for short-term storage.
<br><br>

> Overall, we decided to use `Parquet file` as its the most balanced option from time, size and suitability of this project.